In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
import ast 
import re
import datetime
from datetime import datetime, timedelta
from ast import literal_eval
import warnings; warnings.simplefilter('ignore') # устраним предупреждения

from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [146]:
RANDOM_SEED = 42

In [147]:
!pip freeze > requirments.txt

In [148]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')


In [149]:
df_train['sample'] = 1 # помечаем train
df_test['sample'] = 0 # помечаем test
df_test['Rating'] = 0 # в тесте нет значения Rating, его нужно предсказать

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем в единый датафрейм

In [150]:
df

Основная, но не завершенная предобработка данных

In [151]:

# уберем пробелы из названий столбцов
df.columns = ['Restaurant_id','City','Cuisine_Style','Ranking','Price_Range','Number_of_Reviews','Reviews','URL_TA','ID_TA','sample','Rating']

df['Reviews'] = df['Reviews'].fillna('no_reviews')

pattern = re.compile('\'\d+\/\d+\/\d+\'?') # Создаем шаблон для поиска дат в тексте отзывов

# выводим даты в отзывах в отдельный столбец
df['only_date'] = df['Reviews'].apply(pattern.findall)

# трансформируем содержание столбца в формат datetime
df['only_date'] = df['only_date'].apply(lambda x: sorted([pd.to_datetime(i).date() for i in x]))

# в зависимости от количества дат в столбце only_date создаем два новых столбца для первой и второй дат

def func(row):
    if len(row['only_date']) == 2:
        return row['only_date'][0], row['only_date'][1]
    elif len(row['only_date']) == 1:
        return row['only_date'][0], row['only_date'][0]
    else:
        return np.nan,np.nan
    
df[['date_1', 'date_2']] = df[['only_date']].apply(func, axis=1, result_type='expand')

# создаем столбец delta, определяющий разницу в днях между первым и вторым отзывами
df['delta'] = df['date_2']
for i in df.index: 
    if df['date_2'][i] != np.nan or df['date_1'][i] != np.nan:
        df['delta'][i] = df['date_2'][i] - df['date_1'][i]

# заполним пропуски в delta нулем
df.delta = df.delta.fillna(0)

# формат количества дней переведем в числовой формат, необходимый для обучения модели
def to_integer(dt_time):
    return dt_time.days

for i in df.index: 
    if df.delta[i] != 0: 
        df.delta[i] = to_integer(df.delta[i])

df.delta = df.delta.astype(int)


In [152]:
# анализ ранжирования по цене показал, что подавляющее большинство ресторанов находится в среднем ценовом диапазоне
# поскольку ресторан среднего ценового диапазона всегда находится в позции между дешевым и дорогим ресторанами, 
# и в основном преобладают средние по стоимости рестораны, мы можем разбить все рестораны на категории 1, 2 и 3 и 
# отнести неизвестные по ценовой категории рестораны к средней категории

for i in df.index: 
    if df['Price_Range'][i] == '$':
        df['Price_Range'][i] = 1
    elif df['Price_Range'][i] == '$$$$':
        df['Price_Range'][i] = 3
    else:
        df['Price_Range'][i] = 2

df.Price_Range = df.Price_Range.astype(int)

# применение df.groupby('Restaurant_id')['Restaurant_id'].count() показало, что многие рестораны являются частью 
# ресторанной сети. Это фактор может быть значим для поиска целевой переменной, так как сетевые рестораны,
# как правило, дорожат франшизой и поддерживают единые стандарты качества в обслуживании клиентов.
# Поэтому создаем новый признак - num_of_rest - показатель количества ресторанов в сети 

df['num_of_rest'] = df.groupby('Restaurant_id')['Restaurant_id'].count()
idrest = dict(df.groupby('Restaurant_id')['Restaurant_id'].count())
df.num_of_rest = df[['Restaurant_id', 'num_of_rest']].apply(lambda x: idrest[x[0]] if pd.isna(x[1]) else x[1], axis=1)

# демонстрационный ролик о примениии kaggle дал подсказку, что признак Ranking имеет наиболее важное значение 
# из всех признаков, обсуждаемых в ролике. При этом позиции ресторанов в Ranking датафрейма имеют нормальное 
# распределение по городам. Следовательно, необходимо продумать о соотносимости позиции в Ranking между ресторанами,
# находящимися в разных городах.
# Это наводит на мысль о создании универсального для всех городов признака - coef_of_rest.
# Сначала создадим новый столбец в датафрейме:
df['coef_of_rest'] = df['Ranking']

# cоздадим словарь, в котором будет максимальное количество мест в Ranking по конкретным городам:
dict_ranking = dict(df.groupby("City")['Ranking'].max())
# промежуточно заменим значения Ranking в df['coef_of_rest'] максимальным количеством мест в Ranking 
# по соответствующему городу:
df.coef_of_rest = df[['City', 'Ranking']].apply(lambda x: dict_ranking[x[0]], axis=1) 
# а теперь создадим уникальный для соответствующего ресторана признак, соотносимый со значениями этого признака 
# у других ресторанов независимо от города. Чем ниже значение признака для ресторана, тем он лучше. 
df.coef_of_rest = df.Ranking / df.coef_of_rest

In [153]:
df

In [154]:
display(df.corr()) 

In [155]:
# посмотрим на распределение значений coef-of-rest. Оно нормальное. На каждые 0.1 значения этого коэффициента приходится примерно 10% ресторанов соответствующего города 
df.coef_of_rest.quantile([.10])

In [156]:
df.coef_of_rest.quantile([.20])

In [157]:
df.coef_of_rest.quantile([.30])

In [158]:
df.Number_of_Reviews[df['coef_of_rest'] < 0.1].median()

In [159]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.1) & (df['coef_of_rest'] < 0.2)].median()

In [160]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.2) & (df['coef_of_rest'] < 0.3)].median()

In [161]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.3) & (df['coef_of_rest'] < 0.4)].median()

In [162]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.4) & (df['coef_of_rest'] < 0.5)].median()

In [163]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.5) & (df['coef_of_rest'] < 0.6)].median()

In [164]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.6) & (df['coef_of_rest'] < 0.7)].median()

In [165]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.7) & (df['coef_of_rest'] < 0.8)].median()

In [166]:
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.8) & (df['coef_of_rest'] < 0.9)].median()

In [167]:
df.Number_of_Reviews[df['coef_of_rest'] >= 0.9].median()

Подробное пояснение предыдущих действий.
У нас остались пропуски в столбце Number_of_Reviews. 
Применение таблицы корреляций выявило, что coef_of_rest наибольшим образом влияет на значение Number_of_Reviews. Следовательно, логично заполнить пропуски Number_of_Reviews в зависимости от значений coef_of_rest.
Для этого попробуем выявить зависимость значений coef_of_rest от Number_of_Reviews.

Применение df.coef_of_rest.quantile([.10]), df.coef_of_rest.quantile([.20]), ... df.coef_of_rest.quantile([.90])
показало, что значения coef_of_rest распределены нормально, на каждый квонтиль значений этого коэффициента, 
кратный 0.1,приходится примерно 0.1 значения от соответствующего коэффициента.
Далее для целей заполнения пропусков была выявлена следующая закономерность (применялась медиана, так как она меньше подвержена влиянию выбросов и аномальных значений): 
df.Number_of_Reviews[df['coef_of_rest'] < 0.1].median()
df.Number_of_Reviews[(df['coef_of_rest'] >= 0.1) & (df['coef_of_rest'] < 0.2)].median()
...
df.Number_of_Reviews[df['coef_of_rest'] >= 0.9].median()

Таким образом, для первого квонтиля (df['coef_of_rest'] < 0.1) медианное значение составило 358.
Для второго квонтиля ((df['coef_of_rest'] >= 0.1) & (df['coef_of_rest'] < 0.2)) - 134.
Для третьего квонтиля - 73.
Для четвертого квонтиля - 40.
Для пятого квонтиля - 22.
А вот для остальных квонтилей медианное значение было практически одинаковым от 10 до 13.

Изложенное позволило сделать вывод о целесообразности следующего подхода:

In [168]:
df['Number_of_Reviews'] = df['Number_of_Reviews'].fillna(0) 
for i in df.index: 
    if df['Number_of_Reviews'][i] == 0:
        if df['coef_of_rest'][i] < 0.1:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[df['coef_of_rest'] < 0.1].median()
        elif df['coef_of_rest'][i] < 0.2:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[(df['coef_of_rest'] >= 0.1) 
                                                                & (df['coef_of_rest'] < 0.2)].median()
        elif df['coef_of_rest'][i] < 0.3:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[(df['coef_of_rest'] >= 0.2) 
                                                                & (df['coef_of_rest'] < 0.3)].median()  
        elif df['coef_of_rest'][i] < 0.4:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[(df['coef_of_rest'] >= 0.3) 
                                                                & (df['coef_of_rest'] < 0.4)].median()  
        elif df['coef_of_rest'][i] < 0.5:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[(df['coef_of_rest'] >= 0.4) 
                                                                & (df['coef_of_rest'] < 0.5)].median()
        else:
            df['Number_of_Reviews'][i] = df.Number_of_Reviews[df['coef_of_rest'] >= 0.5].median()  


In [169]:
# уберем пропуски в колонке Cuisine_Style и разобъем виды кухонь на отдельные составляющие 
df['Cuisine_Style'] = df['Cuisine_Style'].fillna("['unknown_style']") 
df['Cuisine_Style'] = df['Cuisine_Style'].apply(literal_eval)

In [170]:
# создадим вспомогательный датафрейм для разложения на отдельные виды кухонь
df_cuisine = pd.DataFrame(zip(df.Cuisine_Style, df.ID_TA, df.coef_of_rest))
df_cuisine.columns = ['Cuisine_Style','ID_TA','coef_of_rest']

In [171]:
# разобъём на отдельные виды кухонь
df_cuisine = df_cuisine.explode('Cuisine_Style')

In [172]:
# посмотрим на самые популярные виды кухонь

df_cuisine["Cuisine_Style"].value_counts().head(30) 

А теперь посмотрим, как распределены самые популярные виды кухонь (то есть представленные не менее чем в 1000 ресторанах) в зависимости от универсального для всех городов признака - coef_of_res

In [173]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    #  ширина Figure
fig.set_figheight(6) # высота Figure
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Vegetarian Friendly'].hist(bins=100)
plt.show()


In [174]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'European'].hist(bins=100)
plt.show()

In [175]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'unknown_style'].hist(bins=100)
plt.show()

In [176]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Mediterranean'].hist(bins=100)
plt.show()

In [177]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Italian'].hist(bins=100)
plt.show()

In [178]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Vegan Options'].hist(bins=100)
plt.show()

In [179]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Gluten Free Options'].hist(bins=100)
plt.show()

In [180]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Bar'].hist(bins=100)
plt.show()

In [181]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'French'].hist(bins=100)
plt.show()

In [182]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Asian'].hist(bins=100)
plt.show()

In [183]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Pizza'].hist(bins=100)
plt.show()

In [184]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Spanish'].hist(bins=100)
plt.show()

In [185]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Pub'].hist(bins=100)
plt.show()

In [186]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Cafe'].hist(bins=100)
plt.show()

In [187]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Fast Food'].hist(bins=100)
plt.show()

In [188]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'British'].hist(bins=100)
plt.show()

In [189]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'International'].hist(bins=100)
plt.show()

In [190]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Seafood'].hist(bins=100)
plt.show()

In [191]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Japanese'].hist(bins=100)
plt.show()

In [192]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Central European'].hist(bins=100)
plt.show()

In [193]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'American'].hist(bins=100)
plt.show()

In [194]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Sushi'].hist(bins=100)
plt.show()

In [195]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Chinese'].hist(bins=100)
plt.show()

In [196]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Portuguese'].hist(bins=100)
plt.show()

In [197]:
fig, ax = plt.subplots()
fig.set_figwidth(12)    
fig.set_figheight(6)
df_cuisine['coef_of_rest'][df_cuisine['Cuisine_Style'] == 'Indian'].hist(bins=100)
plt.show()

Теперь сделаем выводы от просмотра всех представленных выше графиков.

Наиболее популярные кухни разбиты на следующие категории:

1. Gluten Free Options - самое сильное левое смещение на графике, абсолютное большинство ресторанов с этой кухней входит в 15% лучших ресторанов своего города,
Vegan Options - второе по силе левое смещение на графике, абсолютное большинство ресторанов с этой кухней входит в 20% лучших ресторанов своего города,

2. Vegetarian Friendly, European, Seafood, Central European - абсолютное большинство ресторанов с этой кухней входит в 40% лучших ресторанов своего города,

3. Mediterranean, British - большинство ресторанов с этой кухней входит в 50% лучших ресторанов своего города, но левое смещение слабее, чем у первых двух категорий кухонь,

4. Italian, Bar, French, Asian, Spanish, Pub, International, Japanese, Portuguese  - имеют левое смещение, но слабее, чем у предыдущих кухонь

5. Pizza, Cafe, American, Sushi - еле заметное левое смещение,

6. Fast Food, Indian практически равное распределение. 
По умолчанию в эту категорию можно отнести все менее популярные рестораны (менее 1000 ресторанов), как не отвечающие требованию репрезентативности в выборке,  

7. Chinese имеет слабое правое распределение.

8. Обращают на себя внимание рестораны с неизвестной кухней. Отчетливо видно, что их подавляющее большинство входит в последние 50% ресторанов соответствующего города. 

Изложенное позволяет присвоить количество балов каждой категории ресторанов с шагом 1: соответственно первой категории - 8 балов, последней - 1 бал  

In [198]:
df_cuisine['coef_of_cuisine'] = 3
Cuisine_set8 = {'Gluten Free Options','Vegan Options'}
Cuisine_set7 = {'Vegetarian Friendly','European', 'Seafood', 'Central European'}
Cuisine_set6 = {'Mediterranean','British'}
Cuisine_set5 = {'Italian', 'Bar', 'French', 'Asian', 'Spanish', 'Pub', 'International', 'Japanese', 'Portuguese'}
Cuisine_set4 ={'Pizza', 'Cafe', 'American', 'Sushi'}

def func2(x):
    if x in Cuisine_set8:
        return 8
    elif x in Cuisine_set7:
        return 7
    elif x in Cuisine_set6:
        return 6
    elif x in Cuisine_set5:
        return 5
    elif x in Cuisine_set4:
        return 4
    elif x == 'Chinese':
        return 2
    elif x == 'unknown_style':
        return 1
    else:
        return 3
    
df_cuisine['coef_of_cuisine'] = df_cuisine['Cuisine_Style'].apply(func2)


In [199]:
df_cuisine

In [200]:
# создадим вспомогательный словарь по максимальному значению коэффициента кухни для конкретного ресторана
max_coef_cuis = dict(df_cuisine.groupby('ID_TA')['coef_of_cuisine'].max())

# запишем максимальные значения коэффициента в датафрейм 
df_cuisine['coef_of_cuisine'] = df_cuisine[['ID_TA','coef_of_cuisine']].apply(lambda x: max_coef_cuis[x[0]], axis=1)

# теперь запишем значения коэффициентов в основной датафрейм
df['coef_of_cusine'] = 3
df['coef_of_cusine'] = df[['ID_TA','coef_of_cusine']].apply(lambda x: max_coef_cuis[x[0]], axis=1)     
df

In [201]:
df = pd.get_dummies(df, columns=['City',], dummy_na=True)

In [202]:
display(df.corr()) 

In [203]:
df.info()

In [204]:
df1 = df.drop(['Restaurant_id','Cuisine_Style','Ranking','Reviews','URL_TA','ID_TA','only_date','date_1','date_2'],axis=1)
df1

In [205]:
df1.info()

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели, создаём, обучаем и тестируем модель

In [206]:
train_data = df1.query('sample == 1').drop(['sample'], axis=1)
test_data = df1.query('sample == 0').drop(['sample'], axis=1)

In [207]:
y = train_data.Rating.values
X = train_data.drop(['Rating'], axis=1)

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [209]:
# проверим
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [210]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [211]:
# создаем модель
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [212]:
# обучаем модель
model.fit(X_train, y_train)

# используем обученную модель
y_pred = model.predict(X_test)

In [213]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [214]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh') 

In [215]:
test_data = test_data.drop(['Rating'], axis=1) 

In [216]:
sample_submission

In [217]:
predict_submission = model.predict(test_data)

In [218]:
predict_submission

In [219]:
sample_submission['Rating'] = predict_submission

In [220]:
sample_submission.to_csv('submission.csv', index=False)

In [221]:
sample_submission.head(10)